## This notebook merges results from the sourmash-gather output into one sheet with results from all samples.

#### Import all required modules

In [1]:
import pandas as pd
import os as os
import glob as glob
import numpy as np

#### Navigate to your working directory, which should contain both spreadsheets you want to merge

In [16]:
#os.chdir("/Users/nastassia.patin/Desktop/Projects/Lasker 2019/Metagenomes/sourmash/summarized-PacBiomatches/")
os.chdir("/Users/nastassia.patin/Desktop/Projects/Lasker 2019/PacBio/sourmash/GTDB/summarized-gtdb/")

#### Import an example sheet

In [18]:
summary = pd.read_csv("1903c117_50m-3_m64247_gtdb_tax.summarized.csv")
summary.head()

,query_name,rank,fraction,lineage,query_md5,query_filename,f_weighted_at_rank,bp_match_at_rank
0,NaN,superkingdom,0.063931,d__Bacteria,42ee02fc,1903c117_50m-3_m64247_210708_001818.ccs_7g01.f...,0.063931,102529000
1,NaN,superkingdom,0.004029,d__Archaea,42ee02fc,1903c117_50m-3_m64247_210708_001818.ccs_7g01.f...,0.004029,6461000
2,NaN,superkingdom,0.932040,unclassified,42ee02fc,1903c117_50m-3_m64247_210708_001818.ccs_7g01.f...,0.932040,7887000
3,NaN,phylum,0.037139,d__Bacteria;p__Proteobacteria,42ee02fc,1903c117_50m-3_m64247_210708_001818.ccs_7g01.f...,0.037139,59562000
4,NaN,phylum,0.012897,d__Bacteria;p__Bacteroidota,42ee02fc,1903c117_50m-3_m64247_210708_001818.ccs_7g01.f...,0.012897,20684000


In [19]:
summary.shape

(518, 8)

In [20]:
genus = summary[summary['rank'] == 'genus']
genus = genus[['fraction', 'lineage']]

In [21]:
genus.lineage.nunique()

117

In [22]:
genus.head()

,fraction,lineage
133,0.004337,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...
134,0.003804,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...
135,0.003733,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...
136,0.003071,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...
137,0.002939,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...


In [23]:
genus2 = genus.groupby(['lineage']).sum()
genus2.head()

,fraction
lineage,
d__Archaea;p__Thermoplasmatota;c__Poseidoniia_A;o__Poseidoniales;f__Poseidoniaceae;g__MGIIa-K1,0.000154
d__Archaea;p__Thermoplasmatota;c__Poseidoniia_A;o__Poseidoniales;f__Poseidoniaceae;g__MGIIa-L1,0.002315
d__Archaea;p__Thermoplasmatota;c__Poseidoniia_A;o__Poseidoniales;f__Poseidoniaceae;g__MGIIa-L2,0.000125
d__Archaea;p__Thermoplasmatota;c__Poseidoniia_A;o__Poseidoniales;f__Thalassarchaeaceae;g__MGIIb-O2,0.000784
d__Archaea;p__Thermoplasmatota;c__Poseidoniia_A;o__Poseidoniales;f__Thalassarchaeaceae;g__MGIIb-O3,0.000226


### Function to reformat individual data sheet

In [24]:
def format_sourmash_summary(sheet, sample):
    taxon = sheet[sheet['rank'] == 'species'] # select desired level of taxonomic resolution
    taxon = taxon[['bp_match_at_rank', 'lineage']]
    taxon2 = taxon.groupby(['lineage']).sum()
    taxon2['sample'] = sample
    return(taxon2)

### Apply function to all data sheets and combine

In [26]:
summary = []

for file in glob.glob("*tax.summarized.csv"):
    sheet = pd.read_csv(file)
    a, b, c, d, e = file.split('_')
    sample = a + '_' + b
    df = format_sourmash_summary(sheet, sample)
    summary.append(df)

summary = pd.concat(summary)

In [27]:
summary.head()

,bp_match_at_rank,sample
lineage,,
d__Bacteria;p__Actinobacteriota;c__Acidimicrobiia;o__Acidimicrobiales;f__Ilumatobacteraceae;g__Ilumatobacter_A;s__Ilumatobacter_A sp002711735,224000,1903c123_10m-3
d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Flavobacteriales;f__Flavobacteriaceae;g__GCA-002733185;s__GCA-002733185 sp004213605,107000,1903c123_10m-3
d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Flavobacteriales;f__Flavobacteriaceae;g__GCA-002733185;s__GCA-002733185 sp004214175,71000,1903c123_10m-3
d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Flavobacteriales;f__Flavobacteriaceae;g__SGZJ01;s__SGZJ01 sp902515565,134000,1903c123_10m-3
d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Flavobacteriales;f__Flavobacteriaceae;g__UBA3537;s__UBA3537 sp001735715,73000,1903c123_10m-3


#### Pivot table so each sample is a column

In [28]:
summary_pivoted = summary.pivot_table(index='lineage', columns='sample', values='bp_match_at_rank')
summary_pivoted = summary_pivoted.fillna(0)
summary_pivoted.head()

sample,1903c111_10m-3,1903c117_50m-3,1903c118_23m-3,1903c119_11m-3,1903c122_28m-3,1903c123_10m-3,1903c124_15m-3,1903c126_45m-3,1903c127_7m-3,1903c129_26m-3,1903c144_13m-3,Las19c107_10m-3,Las19c135_5m-3,Las19c138_27m-3
lineage,,,,,,,,,,,,,,
d__Archaea;p__Thermoplasmatota;c__Poseidoniia_A;o__Poseidoniales;f__Poseidoniaceae;g__MGIIa-K1;s__MGIIa-K1 sp009936765,0.0,247000.0,0.0,0.0,0.0,0.0,504000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
d__Archaea;p__Thermoplasmatota;c__Poseidoniia_A;o__Poseidoniales;f__Poseidoniaceae;g__MGIIa-L1;s__MGIIa-L1 sp000246735,0.0,1848000.0,0.0,0.0,0.0,0.0,1726000.0,0.0,0.0,0.0,0.0,0.0,0.0,374000.0
d__Archaea;p__Thermoplasmatota;c__Poseidoniia_A;o__Poseidoniales;f__Poseidoniaceae;g__MGIIa-L1;s__MGIIa-L1 sp002172355,0.0,62000.0,0.0,0.0,0.0,0.0,137000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
d__Archaea;p__Thermoplasmatota;c__Poseidoniia_A;o__Poseidoniales;f__Poseidoniaceae;g__MGIIa-L1;s__MGIIa-L1 sp002495535,0.0,0.0,0.0,0.0,0.0,0.0,107000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
d__Archaea;p__Thermoplasmatota;c__Poseidoniia_A;o__Poseidoniales;f__Poseidoniaceae;g__MGIIa-L1;s__MGIIa-L1 sp002502605,0.0,163000.0,0.0,0.0,0.0,0.0,347000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Save the merged table as a csv

In [29]:
summary_pivoted.to_csv("Lasker2019_PacBio_sourmash_species_bpnumbers.csv")

## Import sheet of other sample group and merge

In [31]:
os.chdir("/Users/nastassia.patin/Desktop/Projects/Lasker 2019/Metagenomes/sourmash/summarized-PacBiomatches/")
summary_illumina = pd.read_csv("Lasker2019_Illumina_sourmash_species_bpnumbers.csv")
summary_illumina.head()

,lineage,1903c111_10m-1,1903c117_50m-1,1903c117_50m-2,1903c118_23m-2,1903c119_11m-2,1903c122_28m-1,1903c122_28m-2,1903c123_10m-1,1903c123_10m-2,...,1903c127_7m-1,1903c127_7m-2,1903c129_26m-1,1903c129_26m-2,1903c144_13m-2,Las19c107_10m-1,Las19c107_10m-2,Las19c135_5m-1,Las19c135_5m-2,Las19c138_27m-1
0,d__Archaea;p__Thermoplasmatota;c__Poseidoniia_...,332000.0,61000.0,0.0,0.0,0.0,286000.0,195000.0,0.0,0.0,...,0.0,0.0,363000.0,328000.0,80000.0,143000.0,72000.0,0.0,0.0,590000.0
1,d__Archaea;p__Thermoplasmatota;c__Poseidoniia_...,1794000.0,1657000.0,1426000.0,424000.0,513000.0,1842000.0,1864000.0,1683000.0,1565000.0,...,1588000.0,1532000.0,1776000.0,1843000.0,1318000.0,1748000.0,1681000.0,1339000.0,1115000.0,1749000.0
2,d__Archaea;p__Thermoplasmatota;c__Poseidoniia_...,85000.0,0.0,0.0,0.0,0.0,56000.0,0.0,0.0,0.0,...,0.0,0.0,115000.0,57000.0,0.0,0.0,0.0,0.0,0.0,88000.0
3,d__Archaea;p__Thermoplasmatota;c__Poseidoniia_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,d__Archaea;p__Thermoplasmatota;c__Poseidoniia_...,262000.0,167000.0,92000.0,91000.0,0.0,171000.0,261000.0,186000.0,130000.0,...,0.0,0.0,390000.0,419000.0,0.0,267000.0,212000.0,0.0,0.0,314000.0


In [32]:
summary_pacbio = summary_pivoted.reset_index()
summary_pacbio.head()

sample,lineage,1903c111_10m-3,1903c117_50m-3,1903c118_23m-3,1903c119_11m-3,1903c122_28m-3,1903c123_10m-3,1903c124_15m-3,1903c126_45m-3,1903c127_7m-3,1903c129_26m-3,1903c144_13m-3,Las19c107_10m-3,Las19c135_5m-3,Las19c138_27m-3
0,d__Archaea;p__Thermoplasmatota;c__Poseidoniia_...,0.0,247000.0,0.0,0.0,0.0,0.0,504000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,d__Archaea;p__Thermoplasmatota;c__Poseidoniia_...,0.0,1848000.0,0.0,0.0,0.0,0.0,1726000.0,0.0,0.0,0.0,0.0,0.0,0.0,374000.0
2,d__Archaea;p__Thermoplasmatota;c__Poseidoniia_...,0.0,62000.0,0.0,0.0,0.0,0.0,137000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,d__Archaea;p__Thermoplasmatota;c__Poseidoniia_...,0.0,0.0,0.0,0.0,0.0,0.0,107000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,d__Archaea;p__Thermoplasmatota;c__Poseidoniia_...,0.0,163000.0,0.0,0.0,0.0,0.0,347000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
summary_all = summary_pacbio.merge(summary_illumina, how='outer', on='lineage')
summary_all.head()

,lineage,1903c111_10m-3,1903c117_50m-3,1903c118_23m-3,1903c119_11m-3,1903c122_28m-3,1903c123_10m-3,1903c124_15m-3,1903c126_45m-3,1903c127_7m-3,...,1903c127_7m-1,1903c127_7m-2,1903c129_26m-1,1903c129_26m-2,1903c144_13m-2,Las19c107_10m-1,Las19c107_10m-2,Las19c135_5m-1,Las19c135_5m-2,Las19c138_27m-1
0,d__Archaea;p__Thermoplasmatota;c__Poseidoniia_...,0.0,247000.0,0.0,0.0,0.0,0.0,504000.0,0.0,0.0,...,0.0,0.0,363000.0,328000.0,80000.0,143000.0,72000.0,0.0,0.0,590000.0
1,d__Archaea;p__Thermoplasmatota;c__Poseidoniia_...,0.0,1848000.0,0.0,0.0,0.0,0.0,1726000.0,0.0,0.0,...,1588000.0,1532000.0,1776000.0,1843000.0,1318000.0,1748000.0,1681000.0,1339000.0,1115000.0,1749000.0
2,d__Archaea;p__Thermoplasmatota;c__Poseidoniia_...,0.0,62000.0,0.0,0.0,0.0,0.0,137000.0,0.0,0.0,...,0.0,0.0,115000.0,57000.0,0.0,0.0,0.0,0.0,0.0,88000.0
3,d__Archaea;p__Thermoplasmatota;c__Poseidoniia_...,0.0,0.0,0.0,0.0,0.0,0.0,107000.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,d__Archaea;p__Thermoplasmatota;c__Poseidoniia_...,0.0,163000.0,0.0,0.0,0.0,0.0,347000.0,0.0,0.0,...,0.0,0.0,390000.0,419000.0,0.0,267000.0,212000.0,0.0,0.0,314000.0


In [34]:
summary_all.to_csv("Lasker2019_Illumina_PacBio_sourmash_species_bpnumbers.csv")